# Otimização de Markowitz

## Índice Sharpe

Mede a rentabilidade em relação à volatilidade da carteira. É normalmente apresentado junto com a volatilidade líquida e a rentabilidade líquida.

Se considerarmos o CDI como ativo livre de risco, a fórmula fica:

<div style="text-align: left">

$$
\frac{\text{rentabilidade do ativo} - \text{CDI}}{\text{volatilidade do ativo}}
$$

</div>


Em alguns casos, podemos desconsiderar o ativo livre de risco. Por exemplo:

Se o retorno foi 20% e a volatilidade foi 10%, o Sharpe do investimento é 2.

---

### Fórmula canônica

<div style="text-align: left">

$$
\text{Sharpe} = \frac{R_p - R_f}{\sigma_p}
$$

</div>

* \(R_p\) — retorno médio do ativo ou portfólio  
* \(R_f\) — taxa livre de risco (ex.: CDI, T‑bill)  
* \(\sigma_p\) — volatilidade (desvio padrão) dos retornos de \(R_p\)

Essa fórmula mede o **retorno excedente por unidade de risco total**.

---

### Quando se usa \(R_f = 0\)

| Situação | Por quê? |
| --- | --- |
| Exemplos didáticos | Para simplificar a explicação e a matemática. |
| Séries já em *excess return* | O retorno livre de risco já foi subtraído. |
| Frequência diária ou intradiária | A taxa diária do CDI é desprezível. |
| Ambientes com juro ≈ 0% | Comuns após 2008 em EUA, Europa ou Japão. |
| Estratégias *self-financing* | O retorno já é líquido de funding (ex.: market-neutral). |

---

### Exemplo

* Retorno anual do ativo: **20%**  
* CDI anual: **12%**  
* Volatilidade anual: **10%**

<div style="text-align: left">

$$
\text{Sharpe} = \frac{0.20 - 0.12}{0.10} = 0.8
$$

</div>

*Se \(R_f = 0\), o mesmo caso daria Sharpe = 2.*

---

## Como funciona a otimização

O objetivo da otimização de Markowitz é **encontrar a carteira com o maior índice de Sharpe**.

Por quê?  
Porque quanto maior o Sharpe, **maior o retorno esperado para cada unidade de risco** (volatilidade) assumido. Isso significa melhor eficiência na relação risco-retorno.

---


## Código

In [5]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import scipy.optimize as minimize
import matplotlib.ticker as mticker


### Busca de ativos e preços

In [70]:
start_date = dt.datetime(2022, 1, 1)
end_date = dt.datetime.today()
# end_date = dt.datetime(2025, 10, 1)

# tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
tickers = ['AAPL', 'NVDA', 'GOOGL', 'AMZN']


prices = yf.download(tickers, start=start_date, end=end_date)['Close']
prices


[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,AMZN,GOOGL,NVDA
Date,,,,
2022-01-03,178.879929,170.404495,144.301605,30.068218
2022-01-04,176.609619,167.522003,143.712433,29.238682
2022-01-05,171.911865,164.356995,137.119446,27.555630
2022-01-06,169.042038,163.253998,137.092072,28.128630
2022-01-07,169.209137,162.554001,136.365036,27.199259
...,...,...,...,...
2025-04-11,198.149994,184.869995,157.139999,110.930000
2025-04-14,202.520004,182.119995,159.070007,110.709999
2025-04-15,202.139999,179.589996,156.309998,112.199997


### Retorno logarítmico vs. aritmético
O retorno logarítmico é preferido em análises financeiras porque é simétrico e aditivo ao longo do tempo.

Isso significa que o retorno total de um período pode ser obtido somando os retornos logarítmicos diários.

Já o retorno aritmético (soma de %) não é aditivo devido à composição, o que pode levar a erros em análises de longo prazo.

**Exemplo:**
- Retorno aritmético: 10% + 10% = 20% (errado pois não considera a composição dos retornos dia a pós dia)
- Retorno logarítmico: ln(1+0.10) + ln(1+0.10) = ln(1.1) + ln(1.1) = 2*ln(1.1) = ln(1.1^2) = ln(1.21) = 21% 


### Matriz de covariância
A matriz de covariância (matrix_cov) mede como os retornos de diferentes ativos variam em relação uns aos outros.

Cada elemento da matriz representa a covariância entre dois ativos.

Valores positivos indicam que os ativos tendem a se mover na mesma direção, enquanto valores negativos indicam movimentos opostos.

In [71]:
returns = prices.pct_change().apply(lambda x: np.log(1+x)).dropna() # Retorno logarítimo
returns_mean = returns.mean() # Media dos retornos diários
matrix_cov = returns.cov() # Matriz de covariância dos retornos diários

print("Média dos Retornos Diários:")
print(returns_mean)

print("\nMatriz de Covariância dos Retornos Diários:")
print(matrix_cov)

Média dos Retornos Diários:
Ticker
AAPL     0.000117
AMZN     0.000016
GOOGL    0.000056
NVDA     0.001475
dtype: float64

Matriz de Covariância dos Retornos Diários:
Ticker      AAPL      AMZN     GOOGL      NVDA
Ticker                                        
AAPL    0.000344  0.000262  0.000241  0.000362
AMZN    0.000262  0.000592  0.000340  0.000504
GOOGL   0.000241  0.000340  0.000439  0.000423
NVDA    0.000362  0.000504  0.000423  0.001283


### Simulação de Carteiras Aleatórias (Monte Carlo)

Este trecho de código simula 100.000 carteiras com diferentes combinações de pesos nos ativos, a fim de avaliar seu desempenho em termos de retorno, risco e índice de Sharpe. Abaixo está um resumo do que acontece:

- **Geração de pesos aleatórios:** Para cada carteira, são gerados pesos aleatórios que somam 1 (100% do capital investido).
- **Cálculo do retorno esperado anual:** O retorno esperado da carteira é calculado com base nos retornos médios diários dos ativos, multiplicado por 252 (dias úteis por ano).
- **Cálculo da volatilidade anual:** A volatilidade esperada da carteira é calculada a partir da matriz de covariância dos retornos diários, também anualizada.
- **Cálculo do índice de Sharpe:** Como a taxa livre de risco foi assumida como 0%, o índice de Sharpe é apenas o retorno anual dividido pela volatilidade anual.

A simulação gera os seguintes vetores:
- `vetor_retornos_esperados`: retorno anual esperado de cada carteira
- `vetor_volatilidades_esperadas`: volatilidade anual de cada carteira
- `vetor_sharpe`: índice de Sharpe de cada carteira
- `tabela_pesos`: pesos dos ativos em cada carteira simulada

A barra de progresso da `tqdm` mostra o andamento da simulação.


In [74]:
from tqdm import tqdm  # Importa a biblioteca tqdm para a barra de progresso

numero_carteiras = 100000 # Número total de carteiras simulados
vetor_retornos_esperados = np.zeros(numero_carteiras) # Vetor de retornos esperados 
vetor_volatilidades_esperadas = np.zeros(numero_carteiras) # Vetor de volatilidades esperadas
vetor_sharpe = np.zeros(numero_carteiras) # Vetor de índices de Sharpe
tabela_pesos = np.zeros((numero_carteiras, len(tickers))) # Tabela de pesos dos ativos

# Adiciona a barra de progresso ao loop
for k in tqdm(range(numero_carteiras), desc="Simulando carteiras"):
    pesos = np.random.random(len(tickers)) # Gera pesos aleatórios
    pesos = pesos/np.sum(pesos) # Normaliza os pesos para que a soma seja 1
    tabela_pesos[k, :] = pesos # Adiciona os pesos à tabela de pesos

    # Calcula o retorno esperado e a volatilidade esperada da carteira
    vetor_retornos_esperados[k] = np.sum(returns_mean * pesos) * 252 # Retorno esperado anualizado (252 dias úteis)
    # Calcula a volatilidade esperada da carteira
    vetor_volatilidades_esperadas[k] = np.sqrt(np.dot(pesos.T, np.dot(matrix_cov * 252, pesos))) # Volatilidade esperada anualizada

    # Calcula o índice de Sharpe (assumindo taxa livre de risco de 0%)
    vetor_sharpe[k] = vetor_retornos_esperados[k] / vetor_volatilidades_esperadas[k]

Simulando carteiras: 100%|██████████| 100000/100000 [00:41<00:00, 2415.25it/s]


In [75]:
# Queremos a carteira com maior sharpe
vetor_sharpe 

array([0.3779926 , 0.18539966, 0.44470462, ..., 0.3665988 , 0.20805467,
       0.32980596], shape=(100000,))

In [82]:
max_sharpe_index = vetor_sharpe.argmax() # Índice da carteira com maior índice de Sharpe    
melhor_carteira = tabela_pesos[max_sharpe_index, :] # Pesos da carteira com maior índice de Sharpe
# Retorno esperado da carteira com maior índice de Sharpe   

np.set_printoptions(suppress=True)  # Desativa a notação científica
print("\nCarteira, com maior índice de Sharpe (maior retorno com menor volatilidade):\n")


tabela_resultados = pd.DataFrame({
    'Ticker': tickers,
    'Peso (%)': [f'{peso*100:.2f}%' for peso in melhor_carteira]
})
print(tabela_resultados)

tabela_retornos_esperados_aritmetica = np.exp(vetor_retornos_esperados) - 1 # Retorno esperado aritmético

# print(tabela_retornos_esperados_aritmetica)
print("\nRetorno esperado 1 ano:")
print(f'{tabela_retornos_esperados_aritmetica[max_sharpe_index]*100:.2f}%') # Retorno esperado da carteira com maior índice de Sharpe

# Prints alternativos para exibir os resultados
# print("\nCarteira com maior índice de Sharpe:")
# print (tickers)
# print([f'{peso*100:.2f}%' for peso in melhor_carteira])

# print("\n\nPesos dos Ativos:")
# print(melhor_carteira)

# Ordena os índices do vetor_sharpe em ordem crescente
indices_ordenados = np.argsort(vetor_sharpe)

# O maior índice de Sharpe
max_sharpe_index = indices_ordenados[-1]

# O segundo maior índice de Sharpe
alternativo_max_sharpe_index = indices_ordenados[-1000]

print(f"Maior Sharpe: {tabela_pesos[max_sharpe_index, :]}")
print(f"alternativo maior Sharpe: {tabela_pesos[segundo_max_sharpe_index, :]}")


Carteira, com maior índice de Sharpe (maior retorno com menor volatilidade):

  Ticker Peso (%)
0   AAPL    0.71%
1   NVDA    2.75%
2  GOOGL    3.58%
3   AMZN   92.96%

Retorno esperado 1 ano:
41.38%
Maior Sharpe: [0.00710427 0.02745308 0.03579831 0.92964435]
alternativo maior Sharpe: [0.15336764 0.04049921 0.20757839 0.59855475]


#### Dica avançada:

O queremos mesmo é aperfeiçoar a maneira de projetar a média de retorno futuro (returns_mean).

Uma maneira mais avançada de fazer isso é usar a TIR (Taxa Interna de Retorno) do valuation.

Se você calculcar a TIR das empresas da bolsa através de um valuation decente, você consegue ter o retorno esperado de todas as empresas no futuro e consequentemente a volatilidade esperada também, a partir da volatilidade implícita utilizando opções.

**Contexto: Otimização de Markowitz**

- Na forma clássica da otimização de carteiras de Markowitz, usamos:

- A média histórica dos retornos para estimar o retorno esperado (𝐸[𝑅])

- A matriz de covariância histórica para representar o risco (volatilidade e correlação entre ativos)

- 💡 Mas usar média histórica pode ser muito fraco para prever o futuro — especialmente em mercados instáveis ou em empresas em transformação.

**1. TIR do Valuation como proxy para retorno esperado:**

- A TIR (Taxa Interna de Retorno) de um valuation é a taxa que igualaria o preço atual de uma ação com seus fluxos de caixa futuros esperados.

- Exemplo: Se a ação custa R$ 50 hoje e seu fluxo de caixa projetado indica que ela vai render R$ 60 em 1 ano, a TIR implícita é de 20%.

- Essa TIR pode ser interpretada como o retorno esperado daquele ativo no futuro.

- ✅ Isso dá uma estimativa mais fundamentada economicamente do que a simples média dos retornos passados.

**2. Volatilidade implícita (via opções) como proxy para risco:**

- Em vez de olhar a volatilidade histórica da ação, podemos usar a volatilidade implícita — que é extraída dos preços das opções no mercado.

- A volatilidade implícita reflete a expectativa do mercado quanto à variabilidade dos preços no futuro.

- ✅ Assim, você teria uma matriz de covariância mais condizente com as percepções futuras do mercado.

**3. Quando isso vale a pena?**

- Para investidores institucionais, gestores fundamentalistas ou modelos híbridos (quant + fundamental).

- Quando você tem boas ferramentas para valorar empresas e capturar precificação de opções.

- Em mercados onde o histórico recente não é confiável como proxy do futuro.
